In [1]:
import numpy as np
from matplotlib import pyplot as plt

For the cosserat rod equations, there are two kinematic equations (one related to translations and another related to rotations) and two dynamic equations (again, one related to translations and another related to rotations). So we tackle this problem in increasing levels of complexity via these milestones:

1) first tackle translations (which are easier to implement) by testing the equations on an elastic beam that is fixed on one end and has a small axial force on the one end. Because the beam is elastic, and the axial force is small, the entire beam behaves like a spring. We can then plot look at how much the beam stretches, which should correspond with analytical spring equations. This is milestone 1.

In [33]:
# This class is not completely general: only supports uniform mass and single radius, assumes element ref_lengths of 1
class cosserat_rod:
    # initialize the rod
    def __init__(self, n_elements: int = 5, positions = 0, rod_mass = 1, 
                 rod_radius = 1, G = 20, E = 40, velocities = 0, internal_forces = 0):
        
        '''Set up quantities that describe the rod'''
        self.G = G
        self.E = E
        self.n_elements = n_elements
        self.n_nodes = self.n_elements+1
        self.positions = positions if positions else np.vstack((np.arange(self.n_nodes),np.zeros((2,self.n_nodes))))
        
        self.lengths_bold = self.positions[:, 1:] - self.positions[:, :-1]
        self.lengths_norm = np.linalg.norm(self.lengths_bold, axis=0, keepdims=True)
        self.reference_lengths_bold = self.positions[:, 1:] - self.positions[:, :-1]
        self.reference_lengths_norm = np.linalg.norm(self.reference_lengths_bold, axis=0, keepdims=True)
        
        self.directors = np.zeros((n_elements, 3, 3)) # this shape is less efficient but easier for us to think about
        for idx in range(n_elements):
            self.directors[idx, :, :] = np.eye(3)
        self.directors[:,:,0] = (self.lengths_bold/self.lengths_norm).T
        
        self.masses = rod_mass*np.ones((1,self.n_nodes,))
        self.masses[0,0] *= 0.5
        self.masses[0,-1] *= 0.5
        
        self.radius = rod_radius * np.ones((1,self.n_elements))
        self.areas = np.pi*self.radius**2     
        
        '''Set up quantities that will capture motion'''
        self.velocities = velocities if velocities else np.zeros((3, self.n_nodes)) 
        self.internal_forces = internal_forces if internal_forces else np.zeros((3, self.n_nodes))
        
        self.dilatations = self.lengths_norm / self.reference_lengths_norm
        
        self.shear_stiffness_matrix = np.zeros((n_elements, 3, 3)) # S
        alpha_c = 4.0 / 3.0
        self.shear_stiffness_matrix[:, 0, 0] = alpha_c * self.G * self.areas # S1
        self.shear_stiffness_matrix[:, 1, 1] = alpha_c * self.G * self.areas # S2
        self.shear_stiffness_matrix[:, 2, 2] = self.E * self.areas # S2
        
        self.tangents = self.lengths_bold / self.lengths_norm
        self.shear_stretch_strains = self.dilatations * self.tangents - self.directors[:, 0, :].T
    

    def rotate_rodrigues(t_frame, t_angle, about=[0.0,0.0,1.0], rad=False):
        """Rotates about one of the axes

        Parameters
        ----------
        t_frame : frame/np.array
            If frame object, then t_frame is given by the process function of
            the frame
            Else just a numpy array (vector/collection of vectors) which you
            want to rotate
        t_angle : float
            Angle of rotation, in degrees. Use `rad` to change behavior
        about : list/np.array
            Rotation axis specified in the world coordinates
        rad : bool
            Defaults to False. True indicates that the t_angle is in degrees rather
            than in radians. False indicates radians.

        Returns
        -------
        rot_frame : np.array 
            The rotated frame
        about : np.array
            The vector about which rotate_rodrigues effects rotation. Same as the
            input argument
        """
        # Check if its in radian or degree
        # Default assumed to be degree
        if not rad:
            t_angle = np.deg2rad(t_angle)

        def normalize(v):
            """ Normalize a vector/ matrix """
            norm = np.linalg.norm(v)
            if np.isclose(norm, 0.0):
                raise RuntimeError("Not rotating because axis specified to be zero")
                return v
            return v / norm

        def skew_symmetrize(v):
            """ Generate an orthogonal matrix from vector elements"""
            # Hard coded. Others are more verbose or not worth it
            return np.array([[0.0,-v[2],v[1]],
                             [v[2],0.0,-v[0]],
                             [-v[1],v[0],0.0]])


        # Convert about to np.array and normalize it
        about = normalize(np.array(about))

        # Form the 2D Euler rotation matrix
        c_angle = np.cos(t_angle)
        s_angle = np.sin(t_angle)

        # DS for 3D Euler rotation matrix
        # Composed of 2D matrices
        I = np.eye(3)
        K_mat = skew_symmetrize(about)
        # rot_matrix = I + K_mat @ (s_angle * I + (1-c_angle)* K_mat)
        rot_matrix = I + (s_angle * K_mat + (1.0 - c_angle) * (K_mat @ K_mat))   
        # print(rot_matrix, U_mat)

        if not (np.allclose(rot_matrix, I)):
            # actually do the rotation
            return rot_matrix @ t_frame, about
        else:
            return rot_matrix @ t_frame, about
            # raise RuntimeError("Not rotating because rotation is identity")
            
    # Modified trapezoidal integration
    def modified_diff(self, t_x):        
        temp = np.pad(t_x, (0,1), 'constant', constant_values=(0,0)) # Using roll calculate the diff (ghost node of 0)
        
        return temp[:-1,:]-np.roll(temp,1)[:-1,:]

    def position_verlet(self, dt, x, v, a):
        """Does one iteration/timestep using the Position verlet scheme

        Parameters
        ----------
        dt : float
            Simulation timestep in seconds
        x : float/array-like
            Quantity of interest / position of COM
        v : float/array-like
            Quantity of interest / velocity of COM
        force_rule : ufunc
            A function, f, that takes one argument and
            returns the instantaneous forcing

        Returns
        -------
        x_n : float/array-like
            The quantity of interest at the Next time step
        v_n : float/array-like
            The quantity of interest at the Next time step
        """
        temp_x = x + 0.5*dt*v
        v_n = v + dt * force_rule(temp_x)
        x_n = temp_x + 0.5 * dt * v_n
        return x_n, v_n

    # run the simulation, specifying external conditions ** for now it's just for first benchmark
    def run(self, ext_forces = 0, t_total = 10, dt = 0.1):
        n_iterations = np.ceil(t_total/dt)
        for time_step in range(int(n_iterations)):
            
            positionsHalf = self.positions + 0.5 * dt * self.velocities
            
            internal_forces = np.zeros((3,self.n_elements))
            for elem in range(self.n_elements):
                internal_forces[:,elem] = self.shear_stiffness_matrix[elem,:,:] @ self.shear_stretch_strains[:,elem] / \
                self.dilatations[0,elem]
            mod_dif = self.modified_diff(internal_forces)
            
            print("internal forces:")
            print(internal_forces)
            
            print("dilatations")
            print(self.dilatations)
            
            dvdt = (mod_dif + ext_forces) / self.masses

            self.velocities += dt*dvdt
            self.positions = positionsHalf + 0.5 * dt * self.velocities
            
            self.lengths_bold = self.positions[:, 1:] - self.positions[:, :-1]
            self.lengths_norm = np.linalg.norm(self.lengths_bold, axis=0, keepdims=True)
            
            self.dilatations = self.lengths_norm / self.reference_lengths_norm
            self.tangents = self.lengths_bold / self.lengths_norm
            self.shear_stretch_strains = self.dilatations * self.tangents - self.directors[:, 0, :].T
            
            # constrain the first element to the wall
            self.positions[:,0] = np.zeros((3,))
        return self.positions
            


stretch_case = cosserat_rod()
ext_force = np.zeros((3, stretch_case.n_nodes))
ext_force[0, -1] = 1
stretch_case.run(ext_force)

internal forces:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
dilatations
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
internal forces:
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.82946341]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]
dilatations
[[1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
  

internal forces:
[[ -95.00418386   79.26633936   76.32927174  -87.16206125   78.63849665
   -99.2475797  -725.50274504  -90.52814567   81.04620593  -94.48513744
    70.73752288  -87.0374626   -89.82320984   79.66566169   81.73072136
   -85.55598564   82.50083085   81.94899537  -85.40031028 -127.14027606]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]]
dilatations
[[ 7.46107683 18.57777121 11.2503109  24.73994157 16.30733705  5.41475046
   0.13054743 12.40692627 30.69162533  7.82269086  6.42537177 25.68503234
  1

     0.            0.            0.            0.            0.        ]]
dilatations
[[  6.1240889   26.79860472  26.75983829  34.80672216  19.32201201
   66.12793626  71.10857446  77.9683313   26.62371469 116.45568402
   30.91532813  17.71642763   4.22434364  45.47353428  42.10725369
  125.6664209   93.99158225  40.92104247   8.23779696 123.97369693]]
internal forces:
[[ 76.19232197 -86.48073066  80.33517237 -86.72486495  78.29638639
   82.4995027  -84.97017676  82.70267816  81.02920113 -84.47245483
   80.08886933  81.54608053 -88.63531644  82.02912074 -85.86565108
   83.07910079 -84.7227556   81.85957559 -88.69372214  83.12802132]
 [  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
    0.           0.           0.           0.         

internal forces:
[[ -92.94118297   82.96878388  -84.59249136   80.80311372   80.54240536
    81.71917019  -84.6612918    78.24449163   82.05202548 -515.56273657
    82.16665508   80.94810644  -84.75028128   83.06385848  -84.64964865
    82.66689277  -84.54592217   83.48963692  -84.1866492    82.47118927]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.        ]]
dilatations
[[9.14046274e+00 1.03808805e+02 1.02580030e+02 2.81818129e+01
  2.59095185e+01 4.07344272e+01 9.46097885e+01 1.51457370e+01
  4.86000947e+01 1.39780439

array([[  0.        ,  41.370612  , -18.71946976,   1.65471641,
        -36.17269895,  87.50959662, -66.52991629, -35.9267941 ,
        -17.45892077, -63.78612984, 138.76868543,  32.08389928,
         96.68096443, 138.84784951,   0.41819377, 147.97722195,
         11.79158562, 105.22651102, 174.48971853, 199.94485495,
        147.88609236],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.    